In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F


n_embd=64
dropout=0.5
n_head=4
n_layer=4
device="cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
#@ Dataset for training gpt, we are using is tiny shakespares dataset:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-07-24 16:54:54--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.09s   

2025-07-24 16:54:54 (12.2 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text=f.read()

In [ ]:
chars=sorted(list(set(text)))
vocab=len(chars)

In [ ]:
#@ Creating maps:
string_to_int={ch:i for i, ch in enumerate(chars)}
int_to_string={i:ch for i, ch in enumerate(chars)}

#@ Encoder:
encode=lambda s: [string_to_int[ch] for ch in s]

#@decoder:
decode=lambda l: ''.join([int_to_string[i] for i in l])


In [ ]:
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.query=nn.Linear(n_embd, head_size, bias=False)
    self.key=nn.Linear(n_embd, head_size, bias=False)
    self.value=nn.Linear(n_embd, head_size, bias=False)

    self.dropout=nn.Dropout(p=0.5)

  def forward(self, x):
    B, T, C=x.shape #batch, sequence len, channel/features
    q=self.query(x) #(B, T, C)
    k=self.key(x)   #(B, T, C)

    wei=q @ k.transpose(-2, -1) # B, T, T

    wei=wei.masked_fill(self.tril[:T, :T]==0, float('inf')) # B, T, T
    wei=F.softmax(wei, dim=-1) #along the tokens
    wei=self.dropout(wei)

    v=self.value(x) #B, T, C
    out= wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
    return out

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj=nn.Linear(n_embd, n_embd)
    self.dropout=nn.Dropout(p=0.5)

  def forward(self, x):
    out=torch.cat([h(x) for h in self.heads], dims=-1)
    out=self.dropout(self.proj(out))
    return out

In [ ]:
class FeedForward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    self.net=nn.Sequential(
        nn.Linear(n_embd, 4*n_embd),
        nn.ReLU(),
        nn.Linear(4*n_embd, n_embd),
        nn.Dropout(dropout)
    )

  def forward(self, x):
    return self.net(x)

- Pre-Layer Normalization is a kind of normalization techniques which helps to make training process more stable by eradicating vanishing gradients or exploding gradient
- post layer norm, used in transformer architecture where deep neural nets has gradient issue
- GPT, bert uses pre LN

In [ ]:
class Block(nn.Module):
  def __init__(self,n_heads, n_embd):
    head_size=n_embd//n_heads
    self.mha=MultiHeadAttention(n_heads, head_size)
    self.ln1=nn.LayerNorm(n_embd)
    self.ff=FeedForward(n_embd)
    self.ln2=nn.LayerNorm(n_embd)

  def forward(self, x):
    x=self.mha(self.ln1(x)) + x #prelayer norm
    x=self.ff(self.ln2(x)) + x
    return x

## Bigram  Model:
(prediction based on the previous words)

In [ ]:
class BigramModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size, n_embd) #helps model understand tokens in continuous vector space
    self.positional_embedding_table=nn.Embedding(block_size, n_embd)
    self.blocks= nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
    self.ln_f=nn.LayerNorm(n_embd)
    self.lm_head=nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T=idx.shape #batch, t=no of tokens in each sequence
    tok_emb=self.token_embedding_table(idx) #B, T, C
    pos_emb=self.positional_embedding_table(torch.arange(T, device=device)) #T, C
    x=tok_emb + pos_emb #B, T, C
    x=self.blocks(x) #B, T, C
    x=self.ln_f(x) #B, T, C
    logits=self.lm_head(x) #B, T, vocab_size

    if targets=None: #targets for training
      loss=None

    else:
      B, T, C= logits.shape
      logits=logits.view(B*T, C)
      targets=targets.view(B*T)
      loss=F.cross_entropy(logits, targets)
    return logits, loss
